In [ ]:
# 튜너 사용 준비하기
# fashion_mnist_tuner.ipynb
import os
import datetime
import tensorflow as tf    
import argparse # 외부에서 입력받기 위해 추가
from kubeflow.fairing.kubernetes.utils import mounting_pvc
from tensorflow.python.keras.callbacks import Callback # 추가

class MyModel(object):
    def train(self):
        
        parser = argparse.ArgumentParser()
        parser.add_argument('--node_amount', required=False, type=int, default=128) # 노드 개수
        parser.add_argument('--epoch', required=False, type=int, default=10) # epoch arg 추가        
        parser.add_argument('--dropout_rate', required=False, type=float, default=0.2) # dropout rate
        parser.add_argument('--optimizer', required=False, type=str, default="sgd") # optimizer
        args = parser.parse_args()        
        
        mnist = tf.keras.datasets.fashion_mnist
        (x_train, y_train), (x_test, y_test) = mnist.load_data()

        print("x_train shape:", x_train.shape, "y_train shape:", y_train.shape)
        print("x_test shape:", x_test.shape, "y_test shape:", y_test.shape)

        x_train, x_test = x_train / 255.0, x_test / 255.0

        
        # 외부에서 입력받은 값을 넣도록 args.xx사용
        model = tf.keras.models.Sequential([
            tf.keras.layers.Flatten(input_shape=(28, 28)),
            tf.keras.layers.Dense(args.node_amount, activation='relu'),
            tf.keras.layers.Dropout(args.dropout_rate),
            tf.keras.layers.Dense(10, activation='softmax')
        ])

        model.compile(optimizer=args.optimizer,
                        loss='sparse_categorical_crossentropy',
                        metrics=['acc'])


        date_folder = datetime.datetime.now().strftime("%Y%m%d-%H%M%S") 
        if os.getenv('FAIRING_RUNTIME', None) is None:
            log_dir = "log/fit/" + date_folder
        else:
            log_dir = "/notebook/log/fit/" + date_folder 

        print(f"tensorboard log dir : {log_dir}")

        tensorboard_cb = tf.keras.callbacks.TensorBoard(log_dir=log_dir,
                                                        histogram_freq=1)
        
        # 외부에서 epoch 입력받기
        model.fit(x_train, y_train,
                    verbose=0, # 0으로 변경하면 기존 로그 미출력
                    validation_data=(x_test, y_test),
                    #epochs=10,
                    epochs=args.epoch,
                    callbacks=[LoggingTrain(), # logging callback 추가
                               tensorboard_cb])
        return model
    
# 간단한 print 포맷팅 함수
def p(msg):
    dt_now = datetime.datetime.now()
    strftime = dt_now.strftime('%Y-%m-%dT%H:%M:%SZ')
    print(f"{strftime} {msg}", flush=True)    
    
    
# Tuner 사용을 위한 로그 포맷 변경
class LoggingTrain(Callback): #로그 형태 변경
    """logging for train
    """
    def on_batch_end(self, batch, logs={}):
        if batch % 100 == 0:
            p(f"batch: {batch}")
            p(f"accuracy={logs.get('acc')} loss={logs.get('loss')}")
            
    def on_epoch_begin(self, epoch, logs={}):
        p(f"epoch: {epoch}")

    def on_epoch_end(self, epoch, logs={}):
        p(f"Validation-accuracy={logs.get('val_acc')}")
        p(f"Validation-loss={logs.get('val_loss')}")
        return    
    
if __name__ == '__main__':
    if os.getenv('FAIRING_RUNTIME', None) is None:
        from kubeflow import fairing

        
        # python 파일 대신 notebook(ipynb)파일을 가져다 쓰기.
        fairing.config.set_preprocessor('notebook', 
                                        notebook_file='fashion_mnist_tuner.ipynb')
        
        
        # DOCKERHUB 사용시
        DOCKER_REGISTRY = 'YOURID' # 도커허브아이디

        # CAP HARBOR REGISTRY 사용시
        # DOCKER_REGISTRY = 'cap.dudaji.com:31480/프로젝트 이름'
        
        fairing.config.set_builder(
            'append',
            image_name='fashion-mnist-tuner', 
            base_image='dudaji/cap-jupyterlab:tf2.0-cpu',
            registry=DOCKER_REGISTRY, 
            push=True)
        
        notebook_volume = mounting_pvc(pvc_name="workspace-handson", 
                                        pvc_mount_path="/notebook")

        fairing.config.set_deployer('job',
                                    pod_spec_mutators=[notebook_volume],
                                    cleanup=True) # 잡을 실행후 완료시 잡을 삭제할지의 여부를 결정
        fairing.config.run()
    else:
        remote_model = MyModel()
        remote_model.train()     



In [ ]:
# simple_pipeline.ipynb
def sum_product_pipeline(a: float = 1, b: float = 1):
    sum_product_op(a, b)